# Configuration

In [ ]:
# Model Configuration
CONTEXT_LENGTH = 512  # Fixed context length for chunks
EMBEDDING_DIM = 256  # Dimension of the token embeddings
NUM_HEADS = 8  # Number of attention heads
NUM_LAYERS = 6  # Number of transformer layers
QK_HEAD_DIM = 16  # Dimension of the query and key heads
V_HEAD_DIM = 32  # Dimension of the value head
MLP_DIM = 1024  # Dimension of the hidden layers in the transformer
DROPOUT_RATE = 0.1  # Dropout rate for regularization

# EMBEDDING_DIM = 128  # Dimension of the token embeddings
# NUM_HEADS = 4  # Number of attention heads
# NUM_LAYERS = 1  # Number of transformer layers
# QK_HEAD_DIM = 8  # Dimension of the query and key heads
# V_HEAD_DIM = 16  # Dimension of the value head
# MLP_DIM = 512  # Dimension of the hidden layers in the transformer

# Data Configuration
VOCAB_SIZE = 2048  # Size of the vocabulary
PADDING = True # Whether to pad sequences
PACKING = True # Whether to pack sequences for training

# Training Configuration
SEED = 42  # Random seed for reproducibility
BATCH_SIZE = 128  # Batch size for training
EPOCHS = 20 # Number of epochs to train
SAMPLE_LIMIT = 1000000  # Set to None to process the entire dataset
LR = 0.001  # Learning rate for the optimizer
WEIGHT_DECAY = 0.01  # Weight decay for the optimizer
BETA1 = 0.9  # Beta1 for the Adam optimizer
BETA2 = 0.999  # Beta2 for the Adam optimizer

# File Paths and Labels
TOKENIZER_FILE = "./data/tinystories-tokenizer"
CHUNK_FILE = "./data/chunked_stories"
# LOG_DIR = None
LOG_DIR = './runs/2025-08-26_17-09-10'
DICT_LABEL = 'seq'

In [ ]:
%load_ext autoreload
%autoreload 2

import os
from datetime import datetime
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
device = 'mps' if torch.backends.mps.is_available() else 'cpu'

from datasets import load_dataset
from tokenizers import Tokenizer

from tensorboardX import SummaryWriter

from llm.modules_torch import SmallLanguageModel, loss_fn, print_story
from llm.data import train_tokenizer, chunk_story, create_dict_parameters, encode_story, pack_stories, pretty_json

In [ ]:
params = create_dict_parameters(locals())
LOG_DIR = f'runs/{datetime.now().strftime("%Y-%m-%d_%H-%M-%S")}' if LOG_DIR is None else LOG_DIR
writer = SummaryWriter(log_dir=LOG_DIR)
if len(list(Path(LOG_DIR).glob('events.out.tfevents.*'))) == 1:
    print(f"Logging parameters")
    writer.add_text('Parameters', pretty_json(params))

# Tokenizer

In [ ]:
tokenizer_path = f'{TOKENIZER_FILE}_{VOCAB_SIZE}_{SAMPLE_LIMIT}.json'
if os.path.exists(tokenizer_path):
    tokenizer = Tokenizer.from_file(tokenizer_path)
    sos_token_id = tokenizer.token_to_id('[SOS]')
    eos_token_id = tokenizer.token_to_id('[EOS]')
    pad_token_id = tokenizer.token_to_id('[PAD]')
    print(f"Tokenizer file {tokenizer_path} already exists. Skipping training.")
else:
    dataset = load_dataset("roneneldan/TinyStories", split="train")
    if SAMPLE_LIMIT:
        dataset = dataset.select(range(min(SAMPLE_LIMIT, len(dataset))))
    tokenizer = train_tokenizer(dataset, vocab_size=VOCAB_SIZE, special_tokens=["[UNK]", "[PAD]", "[SOS]", "[EOS]", "\n"])
    tokenizer.save(tokenizer_path)
    print(f"Tokenizer saved to {tokenizer_path}")

tokenizer = Tokenizer.from_file(tokenizer_path)
encoded = tokenizer.encode("Once upon a time, there was a little fox.\nIt lived in a forest and loved to explore.")

print("\n--- Testing the Tokenizer ---")
print("Tokens:", encoded.tokens)
print("IDs:", encoded.ids)
print("Decoded:", tokenizer.decode(encoded.ids, skip_special_tokens=True))

# Chunking

In [ ]:
if PADDING and PACKING:
    chunk_file_path = f'{CHUNK_FILE}_{VOCAB_SIZE}_{CONTEXT_LENGTH}_{SAMPLE_LIMIT}_padding_packing.npz'
elif PADDING:
    chunk_file_path = f'{CHUNK_FILE}_{VOCAB_SIZE}_{CONTEXT_LENGTH}_{SAMPLE_LIMIT}_padding.npz'
elif PACKING:
    chunk_file_path = f'{CHUNK_FILE}_{VOCAB_SIZE}_{CONTEXT_LENGTH}_{SAMPLE_LIMIT}_packing.npz'
else:
    chunk_file_path = f'{CHUNK_FILE}_{VOCAB_SIZE}_{CONTEXT_LENGTH}_{SAMPLE_LIMIT}.npz'
figure_path = f'./figures/histogram_{VOCAB_SIZE}_{SAMPLE_LIMIT}.png'
if os.path.exists(chunk_file_path):
    print(f"Chunk file {chunk_file_path} already exists. Skipping chunking.")

    # display the existing histogram
    plt.imshow(plt.imread(figure_path))
    plt.axis('off')
else:
    # Load the tokenizer
    if not ('dataset' in locals()):
        dataset = load_dataset("roneneldan/TinyStories", split="train")
        if SAMPLE_LIMIT:
            dataset = dataset.select(range(min(SAMPLE_LIMIT, len(dataset))))
    tokenizer = Tokenizer.from_file(tokenizer_path)

    # Process all stories and collect chunks
    all_chunks = []
    unfinished_chunk = []
    num_non_special_tokens = []
    for story in tqdm(dataset["text"], desc="Chunking stories"):
        if PACKING:
            story_chunks, non_special_token_count = encode_story(story, tokenizer, '[SOS]', '[EOS]')
            all_chunks.append(story_chunks)
        else:
            story_chunks, unfinished_chunk, non_special_token_count = chunk_story(story, tokenizer, '[SOS]', '[EOS]', CONTEXT_LENGTH,
                                                            unfinished_chunk=unfinished_chunk, padding=PADDING, pad_token='[PAD]')
            all_chunks.extend(story_chunks)
        num_non_special_tokens.append(non_special_token_count)

    # Convert list to numpy array for efficient storage
    if PACKING:
        chunks_array = np.array(pack_stories(all_chunks, CONTEXT_LENGTH, tokenizer.token_to_id('[PAD]')), dtype=np.int32)
    else:
        chunks_array = np.array(all_chunks, dtype=np.int32)
    unique_tokens, counts = np.unique(chunks_array, return_counts=True)

    # Print statistics
    print(f"Total tokens: {CONTEXT_LENGTH * chunks_array.shape[0]:,}")
    print(f"Total non-special tokens: {np.sum(counts[3:]):,}")
    print(f"Number of special tokens: {np.sum(counts[:3]):,}")
    print(f"Array shape: {chunks_array.shape}")

    # Save the chunks to a compressed file
    print(f"Saving chunks to {chunk_file_path}...")
    np.savez_compressed(chunk_file_path, chunks=chunks_array)
    print(f"Saved successfully! File size: {os.path.getsize(chunk_file_path) / (1024 * 1024):.2f} MB")
    if PADDING and PACKING:
        text_info_path = f'./data/chunk_info_{VOCAB_SIZE}_{CONTEXT_LENGTH}_{SAMPLE_LIMIT}_padding_packing.txt'
    elif PADDING:
        text_info_path = f'./data/chunk_info_{VOCAB_SIZE}_{CONTEXT_LENGTH}_{SAMPLE_LIMIT}_padding.txt'
    elif PACKING:
        text_info_path = f'./data/chunk_info_{VOCAB_SIZE}_{CONTEXT_LENGTH}_{SAMPLE_LIMIT}_packing.txt'
    else:
        text_info_path = f'./data/chunk_info_{VOCAB_SIZE}_{CONTEXT_LENGTH}_{SAMPLE_LIMIT}.txt'

    plt.hist(num_non_special_tokens, bins=50, color='blue')
    plt.title("Distribution of Story Lengths")
    plt.xlabel("Length (number of tokens)")
    plt.ylabel("Frequency")
    plt.savefig(figure_path)

    with open(text_info_path, 'w') as f:
        f.write(f"Sample limit: {SAMPLE_LIMIT:,}\n")
        f.write(f"Vocabulary Size: {VOCAB_SIZE:,}\n")
        f.write(f"Context length: {CONTEXT_LENGTH:,}\n")
        f.write(f"Number of chunks: {chunks_array.shape[0]:,}\n")
        f.write(f"Number of tokens: {CONTEXT_LENGTH * chunks_array.shape[0]:,}\n")
        f.write(f"Number of non-special tokens: {np.sum(counts[3:]):,}\n")
        f.write(f"Number of special tokens: {np.sum(counts[:3]):,}\n")
        f.write(f"Padding used: {PADDING}\n")
        f.write(f"Packing used: {PACKING}\n")

# Data Pipeline

In [ ]:
torch.manual_seed(SEED)
data = np.load(chunk_file_path)['chunks']

# Create dataset
class StoryDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

dataset = StoryDataset(data)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE,
shuffle=True)
num_batches = len(dataloader)
print(f"Number of batches per epoch: {num_batches}")

In [ ]:
for x in dataloader:
    x = x.numpy()
    print(x.shape)
    print(type(x))
    text = tokenizer.decode(x[0], skip_special_tokens=False).split(' ')
    for i in range(0, len(text), 30):
        print(' '.join(text[i:i+30]))
    text = tokenizer.decode(x[BATCH_SIZE-1], skip_special_tokens=False).split(' ')
    for i in range(0, len(text), 30):
        print(' '.join(text[i:i+30]))
    break  # Just to test the first batch

# Model

In [ ]:
model = SmallLanguageModel(vocab_dim=VOCAB_SIZE, embed_dim=EMBEDDING_DIM, n_head=NUM_HEADS, num_layers=NUM_LAYERS, qk_head_dim=QK_HEAD_DIM, v_head_dim=V_HEAD_DIM, mlp_dim=MLP_DIM, max_len=CONTEXT_LENGTH, dropout_rate=DROPOUT_RATE)
model.to(device)
# check number of parameters
print(f"Number of parameters in the model: {sum(param.numel() for param in model.parameters()):,}")
optimizer = optim.AdamW(model.parameters(), lr=LR, betas=[BETA1, BETA2], weight_decay=WEIGHT_DECAY)

# Training

In [ ]:
if os.path.exists(f'{LOG_DIR}/checkpoint.pt'):
    checkpoint = torch.load(f'{LOG_DIR}/checkpoint.pt', weights_only=False)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    torch.set_rng_state(checkpoint['rng_state'])
    start_epoch_from = checkpoint['start_epoch_from']
    print(f"Resuming training from epoch {start_epoch_from}.")
else:
    start_epoch_from = 0

In [ ]:
generate_story_freq = np.ceil(num_batches / 2) + 1
for epoch in range(start_epoch_from, EPOCHS):
    losses = []
    for i, seq in enumerate(dataloader):
        tensor_seq = seq.to(device)
        input_seq = tensor_seq[:, :-1]  # Exclude the last token for input
        target_seq = tensor_seq[:, 1:]  # Exclude the first token for target
        optimizer.zero_grad()
        loss = loss_fn(model, input_seq, target_seq, pad_token_id=pad_token_id if PADDING else None)
        loss.backward()
        optimizer.step()
        print(f"Batch {i}, Loss: {loss.item():.4f}")
        writer.add_scalar('Loss/train', loss.item(), epoch * num_batches + i)
        if (i+1) % generate_story_freq == 0:
            tokens = model.generate('[SOS]', tokenizer, max_length=CONTEXT_LENGTH, eos_token_id=eos_token_id, temp=1.0)
            print_story(tokens, tokenizer)
        losses.append(loss.item())
    #     break
    # break
    avg_loss = np.mean(losses)
    print(f"Epoch {epoch + 1}, Average Loss: {avg_loss:.4f}")
    writer.add_scalar('Loss/epoch_train', avg_loss.item(), epoch)
    tokens = model.generate('[SOS]', tokenizer, max_length=CONTEXT_LENGTH, eos_token_id=eos_token_id, temp=1.0)
    print_story(tokens, tokenizer)
    checkpoint = {
        'start_epoch_from': epoch + 1,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'rng_state': torch.get_rng_state(),
        'loss': avg_loss,
    }
    torch.save(checkpoint, f'{LOG_DIR}/checkpoint.pt')
writer.add_hparams(params, {'hparam/last_loss': avg_loss.item()})